In [78]:
import pandas as pd
import numpy as np

In [79]:
data_test = pd.read_csv('test_data_plus&all.csv')

In [80]:
northesat = ['黑龙江省','吉林省','辽宁省']
huaeast = ['上海市','江苏省','浙江省','安徽省','福建省','江西省','山东省','台湾省']
huanorth = ['北京市','天津市','山西省','河北省','内蒙古自治区']
huacenter = ['湖北省','湖南省','河南省']
huasouth = ['广东省','广西壮族自治区','海南省','香港特别行政区','澳门特别行政区']
southwest = ['四川省','贵州省','云南省','重庆市','西藏自治区']
northwest = ['陕西省','甘肃省','青海省','宁夏回族自治区','新疆维吾尔自治区']

In [ ]:
holiday = ['2018-01-01','2018-02-15','2018-02-16','2018-02-17','2018-02-18','2018-02-19','2018-02-20','2018-02-21']

In [81]:
data_test.drop(['钢材品类','起运地经度','起运地纬度','目的地经度','目的地纬度','start_district','start_city','desti_district','desti_city',
               'start_weather_url','start_weather','start_wind_dire','start_wind_scale','desti_weather_url','desti_max_temp','desti_min_temp',
               'desti_weather','desti_wind_dire','desti_wind_scale','desti_unit_oil_price'],axis=1,inplace=True)

In [82]:
def calc_length(data):
    cate = data['车型'].unique().tolist()
    dd = pd.DataFrame()
    for i in cate:
        cate_data = data[data['车型'] == i]
        cate_data.loc[cate_data['车长'].isnull(),'车长'] = cate_data.loc[data['车长'].notnull()]['车长'].median()
        dd = dd.append(cate_data)
    return dd

In [83]:
data_test = calc_length(data_test)

C:\Users\user\Anaconda2\envs\py35\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [84]:
data_test = data_test.sort_values(by='Unnamed: 0',axis=0)

In [85]:
data_test.drop('Unnamed: 0',axis=1,inplace=True)

In [86]:
data_test = data_test.join(pd.get_dummies(data_test['车型'],prefix='车型_'))
data_test.drop('车型',axis=1,inplace=True)

In [87]:
#将日期处理成季节
def season_transform(date):
    if date[5:7] == '01' or date[5:7] == '02' or date[5:7] == '03':
        x = 1
    elif date[5:7] == '04' or date[5:7] == '05' or date[5:7] == '06':
        x = 2
    elif date[5:7] == '07' or date[5:7] == '08' or date[5:7] == '09':
        x = 3
    elif date[5:7] == '10' or date[5:7] == '11' or date[5:7] == '12':
        x = 4
    return x
date = data_test[['日期']].applymap(season_transform)
date.rename(columns={'日期':'季节'},inplace=True)
data_test = data_test.join(date)
date_dummies = pd.get_dummies(data_test['季节'])
date_dummies.rename(columns={1:'春',2:'夏',3:'秋',4:'冬'},inplace=True)
data_test = data_test.join(date_dummies)
data_test.drop('季节',axis=1,inplace=True)
data_test.drop('日期',axis=1,inplace=True)

In [11]:
#将日期处理成节日
def date_transform(date):
    if date in holiday:
        x = 1
    else:
        x = 0
    return x
date = data_test[['日期']].applymap(date_transform)
date.rename(columns={'日期':'节日'},inplace=True)
data_test = data_test.join(date)
data_test.drop('日期',axis=1,inplace=True)

In [90]:
data_test = data_test.join(pd.get_dummies(data_test['一级品类']))
data_test.rename(columns={1:'品类1',2:'品类2',3:'品类3',4:'品类4'},inplace=True)

In [91]:
data_test.drop('一级品类',axis=1,inplace=True)

In [92]:
def province_transform(province):
    if province in northesat:
        x = 1
    elif province in huaeast:
        x = 2
    elif province in huanorth:
        x = 3
    elif province in huacenter:
        x = 4
    elif province in huasouth:
        x = 5
    elif province in southwest:
        x = 6
    elif province in northwest:
        x = 7
    return x

In [93]:
start_district = data_test[['start_province']].applymap(province_transform)
start_district.rename(columns={'start_province':'start_district'},inplace=True)
data_test = data_test.join(start_district)

In [94]:
desti_district = data_test[['desti_province']].applymap(province_transform)
desti_district.rename(columns={'desti_province':'desti_district'},inplace=True)
data_test = data_test.join(desti_district)

In [95]:
start_district_dummies = pd.get_dummies(data_test['start_district'],prefix='start_district_')
desti_district_dummies = pd.get_dummies(data_test['desti_district'],prefix='desti_district_')

In [96]:
data_test = pd.concat([data_test,start_district_dummies,desti_district_dummies],axis=1)

In [97]:
data_test.drop(['start_province','desti_province','start_district','desti_district'],axis=1,inplace=True)

In [98]:
mean_temp = []
for i in range(len(data_test)):
    temp = (float(data_test['start_max_temp'][i]) + float(data_test['start_min_temp'][i])) / 2
    mean_temp.append(temp)
mean_t = pd.DataFrame(mean_temp)

In [99]:
mean_t.rename(columns={0:'平均温度'},inplace=True)

In [100]:
data_test = data_test.join(mean_t)
data_test.drop(['start_max_temp','start_min_temp'],axis=1,inplace=True)

In [101]:
std_oil_value = data_test[['start_unit_oil_price']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_oil_value.rename(columns={'start_unit_oil_price':'标准化油价'},inplace=True)
data_test = data_test.join(std_oil_value)
std_value = data_test[['货值（吨/元）']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_value.rename(columns={'货值（吨/元）':'标准化货值（吨/元）'},inplace=True)
data_test = data_test.join(std_value)
std_dist = data_test[['运距']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_dist.rename(columns={'运距':'标准化运距'},inplace=True)
data_test = data_test.join(std_dist)
std_car_size = data_test[['车长']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_car_size.rename(columns={'车长':'标准化车长'},inplace=True)
data_test = data_test.join(std_car_size)
std_temp = data_test[['平均温度']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
std_temp.rename(columns={'平均温度':'标准化平均温度'},inplace=True)
data_test = data_test.join(std_temp)
data_test.drop(['start_unit_oil_price','货值（吨/元）','运距','车长','平均温度'],axis=1,inplace=True)

In [102]:
data_test.to_csv('test_features(season).csv')